# A full business solution
## Now we will take our project from Day 1 to the next level
## BUSINESS CHALLENGE:
Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
#imports 

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key=os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? ")

MODEL='gpt-4o-mini'
openai=OpenAI()


There might be a problem with your API key? 


In [3]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed=Website("https://www.anthropic.com/")
print(ed.get_contents)
ed.links

<bound method Website.get_contents of <__main__.Website object at 0x0000027224849150>>


['/',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/3-5-models-and-computer-use',
 '/claude/sonnet',
 '/claude/haiku',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosure-policy',
 'https://trust.anthropic.com/']

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt="You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt+="You should respond in JSON as in this example:"
link_system_prompt+="""
{
    "links":[
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links":[
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.anthropic.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/claude
/team
/enterprise
/api
/pricing
/research
/company
/careers
/news
https://www.anthropic.com/research#entry:8@1:url
https://www.anthropic.com/claude
https://claude.ai/
/api
/news/3-5-models-and-computer-use
/claude/sonnet
/claude/haiku
/news/claude-for-enterprise
/research/constitutional-ai-harmlessness-from-ai-feedback
/news/core-views-on-ai-safety
/jobs
/
/claude
/api
/team
/pricing
/research
/company
/customers
/news
/careers
mailto:press@anthropic.com
https://support.anthropic.com/
https://status.anthropic.com/
/supported-countries
https://twitter.com/AnthropicAI
https://www.linkedin.com/company/anthropicresearch
https://www.youtube.com/@anthropic-ai
/legal/consumer-terms
/legal

In [9]:
def get_links(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type":"json_object"}
    )
    result=response.choices[0].message.content
    return json.loads(result)

In [10]:
anthropic=Website("https://www.anthropic.com/")
anthropic.links

['/',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/3-5-models-and-computer-use',
 '/claude/sonnet',
 '/claude/haiku',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosure-policy',
 'https://trust.anthropic.com/']

In [11]:
get_links("https://www.anthropic.com/")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result="Landing page:\n"
    result+=Website(url).get_contents()
    links=get_links(url)
    print("Found links:",links)
    for link in links["links"]:
        result+=f"\n\n{link['type']}\n"
        result+=Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Learn more
Announcements
Introducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku
Oct 22, 2024
Model updates
3.5 Sonnet
3.5 Haiku
Our Work
Product
Claude for En

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}


"You are looking at a company called: Anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nClaude\nOverview\nTeam\nEnterprise\nAPI\nPricing\nResearch\nCompany\nCareers\nNews\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.5 Sonnet\nClaude 3.5 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nStart using Claude to drive efficiency and create new revenue streams.\nLearn more\nAnnouncements\nIntroducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku\nOct 22, 2024\nModel updates\n3.5 Sonnet\n3.5 Haiku\nOur Work\nProduct\nClaude for Enterprise\nSep 4, 2024\nAlignment\n·\nResearch\nConstitutional AI: Harmlessness from AI Feedback\nDec 15, 2022\nAnnouncements\nCore Views on AI Safety: When, Why, What, and How\nMar 8, 2023\nWo

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}


# Anthropic Company Brochure

## Overview

Welcome to Anthropic, an AI safety and research company based in San Francisco. Our mission is to create reliable, interpretable, and steerable AI systems that can enhance human capabilities while prioritizing safety. We strive to understand the complexities and potential risks associated with AI and share our insights to benefit society.

## Our Product: Claude

Meet **Claude**, our cutting-edge AI model designed to streamline workflows and enhance productivity across various sectors. With features tailored for enterprise solutions, Claude empowers teams to collaborate efficiently, harnessing shared knowledge to improve decision-making and outputs.

### Key Offerings:
- **Claude 3.5 Sonnet**: Our most intelligent AI model.
- **Claude for Enterprise**: Integrating organizational knowledge to foster innovation and efficiency.

## Company Culture

At Anthropic, we cultivate a collaborative and inclusive environment where diverse perspectives are valued. Our culture is a blend of disciplines—ranging from machine learning and physics to policy making and business—that come together under common goals. 

### Our Values
1. **Mission-Driven**: Focused on building AI systems that help society thrive.
2. **Trust and Openness**: Fostering high levels of trust within the team, where honesty and emotional maturity are paramount.
3. **Collaboration**: Encouraging teamwork across various sections, sharing insights and supporting each other’s growth.
4. **Pragmatism**: Valuing practical solutions over complexity and emphasizing empirical approaches in research and engineering.

## Research & Safety Focus

At Anthropic, safety is treated as a systematic science. We conduct frontier research, apply varied safety techniques, and engage in partnerships to understand and mitigate potential risks tied to AI systems. Our research informs our product development, ensuring that safety remains at the forefront.

## Careers

Join our interdisciplinary team dedicated to making AI safe and reliable. We offer a supportive environment that prioritizes employee wellness, with comprehensive benefits including:

- Health, dental, and vision insurance.
- Flexible paid time off and 22 weeks of paid parental leave.
- Competitive salaries paired with equity packages.
- Education stipends and a wellness stipend.
  
### Hiring Process
Our hiring process focuses on identifying candidates who bring unique strengths from diverse backgrounds. We value direct evidence of ability, whether it comes from independent research or contributions to open-source software. All interviews are conducted remotely to accommodate candidates worldwide.

## Customers & Partnerships

We collaborate with a range of stakeholders—including businesses, nonprofits, and civil society groups—to ensure safe AI deployment across industries. Our solutions aim to transform how teams work, fostering productivity and innovation while maintaining data security.

## Join Us

Are you ready to contribute to the future of AI? Explore current job openings on our [Careers Page](https://www.anthropic.com/careers) and become part of a team that values trust, collaboration, and innovation.

[Explore more about Anthropic](https://www.anthropic.com)

---

For further inquiries, please feel free to connect with us through our social media channels:
- [Twitter](https://twitter.com/AnthropicAI)
- [LinkedIn](https://linkedin.com/company/anthropic)
- [YouTube](https://youtube.com/c/Anthropic) 

Join us in making a meaningful impact in the world of AI!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream= openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    response=""
    display_handle=display(Markdown(""),display_id=True)
    for chunk in stream:
        response +=chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown","")
        update_display(Markdown(response),display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "http://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face Brochure

## Welcome to Hugging Face

**Hugging Face is the AI community building the future.** Our platform fosters collaboration among machine learning enthusiasts and professionals as we collectively create and share models, datasets, and applications.

---

## Our Mission

At Hugging Face, we are on a mission to **democratize good machine learning**—one commit at a time. We provide an open-source stack that empowers individuals and organizations to create AI solutions that matter. 

---

## Why Choose Us?

- **Collaboration in ML:** Join a **vibrant community** of over 50,000 organizations, including tech giants like Google, Amazon Web Services, Microsoft, and many more.
- **Explore Diverse Modalities:** Whether you are working on text, image, video, audio, or 3D projects, our platform allows you to explore them all.
- **Cutting-edge Tools:** Utilize our state-of-the-art libraries such as **Transformers** for ML in PyTorch, TensorFlow, and JAX, and **Diffusers** for revolutionary image and audio generation.

---

## Company Culture

Hugging Face champions an open and inviting culture where **innovation thrives**. Our team's values include:

- **Community-Driven:** Contributions from members around the world foster a collective intelligence that drives our advancements.
- **Inclusivity:** We believe in building a diverse team where every voice matters.
- **Passion for Learning:** Our team is always encouraged to learn new technologies and methodologies, fostering personal and professional growth.

---

## Join Our Team

Are you passionate about AI and machine learning? We are always looking for **talented individuals** to join our team! 

- **Current Openings:** We offer a variety of career opportunities spanning several specializations.
- **Work Flexibility:** We promote balance in work and life with remote working options.
- **Growth Opportunities:** Learn and grow in a fast-paced environment focused on cutting-edge technology.

---

## Customer Success

With more than **50,000 organizations** already leveraging Hugging Face, we cater to various industries—from healthcare to gaming, facilitating their journey in AI and machine learning.

- **Enterprise Solutions:** We provide dedicated support and enterprise-grade solutions, enabling businesses to run their AI applications with confidence.

---

## Connect with Us

Join us today and be part of the AI revolution! You can connect with us through:

- **Website:** [Hugging Face](https://huggingface.co)
- **Social Media:** Follow us on [Twitter](https://twitter.com/huggingface), [LinkedIn](https://linkedin.com/company/huggingface), and [Discord](https://discord.com/invite/huggingface).

---

Together, let's build the future of AI!



# Convert the brochure to a specified language


In [21]:
def create_brochure_language(company_name, url, language):
    language_prompt = f"You are a professional translator and writer specializing in creating and translating brochures. Convert the brochure to {language} while maintaining its original tone, format, and purpose."
    user_language_prompt = f"Generate a brochure for the company '{company_name}' available at the URL: {url}, and translate it into {language}."
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": language_prompt},
            {"role": "user", "content": user_language_prompt}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))


In [22]:
create_brochure_language("HuggingFace", "http://huggingface.co","Hindi")

### HuggingFace Brochure

---

#### **Welcome to HuggingFace!**

**Transforming the Way You Interact with AI**

At HuggingFace, we believe in making state-of-the-art AI accessible to everyone. Our mission is to democratize AI and enable a future where building AI applications is simple and intuitive.

---

### **Our Core Products**

1. **Transformers Library**
   - A state-of-the-art library for Natural Language Processing (NLP) with thousands of pre-trained models.
   - Easily integrate models into your applications and explore cutting-edge functionalities.

2. **Datasets**
   - A comprehensive collection of datasets to streamline your machine learning workflows.
   - Access to a wide variety of data sources for training and refining your models.

3. **Spaces**
   - An interactive platform to showcase your models and applications.
   - Share your work with the community and collaborate on exciting AI projects.

---

### **Why Choose HuggingFace?**

- **Community Driven:** Join a thriving community of AI enthusiasts and professionals.
- **Open Source:** All our tools and resources are open-source, encouraging innovation and collaboration.
- **Easy to Use:** Designed for developers of all skill levels, our tools support quick learning and implementation.

---

### **Get Started Today!**

Visit us at [huggingface.co](http://huggingface.co) to explore our offerings, access tutorials, and join our community.

---

### **Contact Us**

**Email:** contact@huggingface.co  
**Twitter:** @huggingface  
**GitHub:** huggingface

---

#### **Thank You!**

Together, let’s shape the future of AI!

---

### **हगिंगफेस में आपका स्वागत है!**

**एआई के साथ आपके इंटरैक्शन के तरीके को बदलना**

हगिंगफेस में, हम मानते हैं कि अत्याधुनिक एआई को सभी के लिए सुलभ बनाना चाहिए। हमारा मिशन एआई का लोकतंत्रीकरण करना और एक ऐसे भविष्य को सक्षम करना है जहां एआई एप्लिकेशन बनाना सरल और सहज हो।

---

### **हमारे मुख्य उत्पाद**

1. **ट्रांसफार्मर्स लाइब्रेरी**
   - प्राकृतिक भाषा प्रोसेसिंग (NLP) के लिए एक अत्याधुनिक लाइब्रेरी जिसमें हजारों प्री-ट्रेंड मॉडल्स हैं।
   - अपने एप्लिकेशनों में मॉडलों को आसानी से समाकलित करें और अत्याधुनिक कार्यात्मकताओं का अन्वेषण करें।

2. **डेटासेट्स**
   - मशीन लर्निंग वर्कफ़्लोज़ को सरल बनाने के लिए एक व्यापक डेटासेट संग्रह।
   - अपने मॉडल को प्रशिक्षित और विकसित करने के लिए विभिन्न डेटा स्रोतों तक पहुँच।

3. **स्पेसेस**
   - अपने मॉडल और एप्लिकेशनों को प्रदर्शित करने के लिए एक इंटरएक्टिव प्लेटफार्म।
   - समुदाय के साथ अपने काम को साझा करें और रोमांचक एआई परियोजनाओं पर सहयोग करें।

---

### **हगिंगफेस को क्यों चुनें?**

- **समुदाय संचालित:** एआई प्रेमियों और पेशेवरों के एक जीवंत समुदाय में शामिल हों।
- **ओपन-सोर्स:** हमारे सभी उपकरण और संसाधन ओपन-सोर्स हैं, जो नवाचार और सहयोग को प्रोत्साहित करते हैं।
- **इस्तेमाल करने में आसान:** सभी कौशल स्तरों के डेवलपर्स के लिए डिज़ाइन किया गया, हमारे उपकरण त्वरित अध्ययन और कार्यान्वयन का समर्थन करते हैं।

---

### **आज ही शुरू करें!**

हमारे उत्पादों का अन्वेषण करने, ट्यूटोरियल्स तक पहुँचने और हमारे समुदाय में शामिल होने के लिए [huggingface.co](http://huggingface.co) पर जाएँ।

---

### **हमसे संपर्क करें**

**ईमेल:** contact@huggingface.co  
**ट्विटर:** @huggingface  
**गिटहब:** huggingface

---

#### **धन्यवाद!**

आइए, हम मिलकर एआई के भविष्य को आकार दें!